In [1]:
from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
from datetime import datetime, date

## Group Assignment
### Team Number: 11
### Team Member Names: Jeffrey, Ray, Baldeep
### Team Strategy Chosen: RISKY (RISKY OR SAFE)

# Initialization of Variables

In [2]:
# List of Tickers to Choose From
ticker_list = pd.read_csv("Tickers.csv")

# Initial Capital (CAD)
capital = 750000

# Trading Fee
trading_fee = 4.95

# Restrictions
avg_month_vol = 150000
min_month_days = 18
currency = ['USD', 'CAD']

# Number of stocks in our Portfolio
num_stocks = 10

# Minimum weight for each stock in the portfolio (in decimals)
min_weight = 1 / (2 * num_stocks)

# Maximum weight for each stock in the portfolio (in decimals)
max_weight = 0.2

# Day that we will purchase stocks for our portfolio
purchase_date = '2023-11-25'

In [3]:
# Portfolios
Portfolio_Final = pd.DataFrame()
Stocks_Final = pd.DataFrame()

# 2. Filtering the tickers

# Before creating our portfolio of stocks, we must first create a filter function to filter out unwanted tickers for this assignment. 

We have to consider:
1) Are the stocks delisted?
2) Are the stocks denominated in USD or CAD?
3) Do the months have at least 18 trading days?
4) Do the stocks have an average monthly volume of at least 150 000 shares? 


1) To do this, we loop through every ticker in the ticker_lst and check if calling fast_info produces an error. If it produces an error, we know the stock is delisted, so we don't add that ticker to the new ticker list. 

2) We get the fast_info of the stock and check if the currency is either USD or CAD. If it is, we add it to the new ticker list. 

3) We use a stock index from the TSX and another from the NYSE and concat the historical data together such that only the days that appear in both are kept. Then, we loop through every month and count the total number of days. If the number of days is less than 18, then we add it to a list and remove the month from the dataframes later. 

4) Using the historical data, we calculate the average monthly volume for each stock by looping through all the valid stocks and then using the mean function. If the average monthly volume is less than 150 000, then we remove the stock from the stock list. 

In [5]:
#define start date and end date variables
start_date = '2023-01-01'
end_date = '2023-10-31'

#define holder variables
stock_hist_data = []
invalid_months = []

#produce a list of all the tickers
ticker_lst = ticker_list[ticker_list.columns[0]]

#filter stocks variable 
def filter_stocks(ticker_lst):
    new_ticker_lst=[]
    
    #loop through every ticker in the ticker_lst and check if it is listed and has a currency of USD or CAD
    for cur_ticker in ticker_lst:
        info = yf.Ticker(cur_ticker).fast_info
        try:
            #check if the currency is USD or CAD
            if(info.currency == 'USD' or info.currency == 'CAD'):
                new_ticker_lst.append(cur_ticker)
        except:
            #output a statement if the stock is delisted
            print(f"{cur_ticker} is not a valid ticker")
    
    #get the historical data of each ticker and add it to a list
    for cur_ticker in new_ticker_lst:
        temp_ticker = yf.Ticker(cur_ticker)
        hist = temp_ticker.history(start=start_date, end=end_date, interval = '1mo')
        stock_hist_data.append(hist)
    
    cad_index = '^GSPTSE' #S&P/TSX Composite Index (CAD)
    usa_index = '^GSPC' #S&P500 Composite Index (USD)
    
    #loop through every month and check if the month has less than 18 trading days
    for cur_month in range(1, 11):
        #get the historical dataframes of the two indexes
        cad_days = yf.Ticker(cad_index).history(start = str(date(2023, cur_month, 1)), end = str(date(2023, cur_month+1, 1)))
        usd_days = yf.Ticker(usa_index).history(start = str(date(2023, cur_month, 1)), end = str(date(2023, cur_month+1, 1)))
        #merge the two dataframes and only keep the duplicates
        total_days = cad_days.reindex(usd_days.index)
        num_days = len(total_days)
        if num_days < min_month_days:
            invalid_months.append(cur_month)
            
    #remove the months that have less than 18 trading days
    for i in range(len(stock_hist_data)):
        cur_hist = stock_hist_data[i]
        for month in invalid_months:
            if month < 10:
                cur_hist.filter(like!=f'2023-0{month}', axis=0)
            else:
                cur_hist.filter(like != f'2023-{month}', axis=0)
    #Calculate the average monthly volume for each stock and remove the stock if it is less than 150 000
    for i in range(len(new_ticker_lst)): 
        cur_ticker = new_ticker_lst[i]
        cur_hist = stock_hist_data[i]
        avgMonthlyVolume = cur_hist['Volume'].mean()
        #check if the average monthly volume is less than 150 000 and remove the stock if it is
        if avgMonthlyVolume < avg_month_vol:
            new_ticker_lst.remove(cur_ticker)
            stock_hist_data.remove(cur_hist)
    return new_ticker_lst

#call the function
filtered_ticker_lst = filter_stocks(ticker_lst)
filtered_ticker_lst

AGN is not a valid ticker
CELG is not a valid ticker
MON is not a valid ticker
RTN is not a valid ticker


['ABBV',
 'ABT',
 'ACN',
 'AIG',
 'AMZN',
 'AXP',
 'BA',
 'BAC',
 'BIIB',
 'BK',
 'BLK',
 'BMY',
 'C',
 'CAT',
 'CL',
 'KO',
 'LLY',
 'LMT',
 'MO',
 'MRK',
 'PEP',
 'PFE',
 'PG',
 'PM',
 'PYPL',
 'QCOM',
 'RY.TO',
 'SHOP.TO',
 'T.TO',
 'TD.TO',
 'TXN',
 'UNH',
 'UNP',
 'UPS',
 'USB']

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.